# Results

In [1]:
import wandb
import pandas as pd
import numpy as np
from datetime import datetime
import yaml

In [29]:
def get_runs(entity="jccamml", project="L65-project", algorithm="find_maximum_subarray_kadane"):
    api = wandb.Api(timeout=30)
    wandb_filters = {
        #"config.algorithms": [algorithm],
        "created_at": {"$gt": "2025-03-22T01"}, # greater than
        #"state": "finished",
    }
    runs = api.runs(f"{entity}/{project}", filters=wandb_filters) 
    print(f"Found {len(runs)} runs.")
    data = {}
    for run in runs:
        if run.config['algorithms'] != [algorithm] or run.summary.get('_step',0) < 80000:
            print(f"Skipping run {run.id} with algorithm {run.config['algorithms']} and steps {run.summary.get('_step',0)}")
            continue 
        
        if data.get((run.config['processor_type'], run.config['hint_teacher_forcing'], run.config['seed'])) is None:
            data[(run.config['processor_type'], run.config['hint_teacher_forcing'], run.config['seed'])]  = run

    return data

In [35]:
# Few seconds
data = get_runs()

Found 147 runs.
Skipping run mf2en0ra with algorithm ['bfs'] and steps 0
Skipping run f8aowemb with algorithm ['bfs'] and steps 1181
Skipping run wskeew1n with algorithm ['bfs'] and steps 0
Skipping run ah31c565 with algorithm ['bfs'] and steps 0
Skipping run nbxcff2v with algorithm ['bfs'] and steps 0
Skipping run wyl28e32 with algorithm ['bfs'] and steps 0
Skipping run c40nf94c with algorithm ['bfs'] and steps 0
Skipping run r0tdmrsk with algorithm ['bfs'] and steps 0
Skipping run l4hssvng with algorithm ['bfs'] and steps 2
Skipping run qv2dvuaj with algorithm ['bfs'] and steps 3
Skipping run 140lw5y2 with algorithm ['find_maximum_subarray_kadane'] and steps 0
Skipping run 6q5qu1tn with algorithm ['find_maximum_subarray_kadane'] and steps 0
Skipping run 7o8sf2j5 with algorithm ['find_maximum_subarray_kadane'] and steps 0
Skipping run chz6czcc with algorithm ['find_maximum_subarray_kadane'] and steps 0
Skipping run drsfg7y7 with algorithm ['find_maximum_subarray_kadane'] and steps 0
S

In [36]:
for c in sorted(data.keys()):
    print(c)

('differential_mpnn_maxmax', 0, 0)
('differential_mpnn_maxmax', 0, 1)
('differential_mpnn_maxmax', 0, 2)
('differential_mpnn_maxmax-ablate_outer', 0, 0)
('differential_mpnn_maxmax-ablate_outer', 0, 1)
('differential_mpnn_maxmax-ablate_outer', 0, 2)
('differential_mpnn_maxmax-ablate_subtract', 0, 0)
('differential_mpnn_maxmax-ablate_subtract', 0, 1)
('differential_mpnn_maxmax-ablate_subtract', 0, 2)
('differential_mpnn_maxmax-sum', 0, 0)
('differential_mpnn_maxmax-sum', 0, 1)
('differential_mpnn_maxmax-sum', 0, 2)
('differential_mpnn_msgdiff', 0, 0)
('differential_mpnn_msgdiff', 0, 1)
('differential_mpnn_msgdiff', 0, 2)
('mpnn', 0, 0)
('mpnn', 0, 1)
('mpnn', 0, 2)
('mpnn', 0.5, 0)
('mpnn', 0.5, 1)
('mpnn', 0.5, 2)


# Analysis - Kadane

In [46]:
cache = {}



In [53]:
algorithm = "find_maximum_subarray_kadane"
results = list()
for (name, run) in data.items():
    history_data = run.history(keys=[f'test_score_current_model_{algorithm}'])  # Fetch full history
    history_data = history_data[(history_data["_step"] <= 80000) & (history_data["_step"] % 500 == 0)]  # Keep only <= 80k steps
    if len(history_data) == 161:
        cache[name] = history_data
    results.append(
        {
        "method": f"{name[0]}{"" if name[1]==0 else "-TeacherForced"}",
        "Train Steps": 40000,
        "Seed": name[2],
        "OOD Accuracy": history_data.iloc[61:81][f'test_score_current_model_{algorithm}'].mean()
        }
    )
    results.append(
        {
        "method": f"{name[0]}{"" if name[1]==0 else "-TeacherForced"}",
        "Train Steps": 80000,
        "Seed": name[2],
        "OOD Accuracy": history_data.iloc[-20:][f'test_score_current_model_{algorithm}'].mean()
        }
    )

In [54]:
pd.DataFrame(results).groupby(["method", "Train Steps"])['OOD Accuracy'].mean().reset_index().pivot_table(index='method', columns='Train Steps', values='OOD Accuracy')

Train Steps,40000,80000
method,,
differential_mpnn_maxmax,0.694255,0.725342
differential_mpnn_maxmax-ablate_outer,0.689559,0.709937
differential_mpnn_maxmax-ablate_subtract,0.644670,0.687150
differential_mpnn_maxmax-sum,0.704517,0.702173
differential_mpnn_msgdiff,0.669808,0.699902
mpnn,0.669784,0.711483
mpnn-TeacherForced,0.729915,0.749862


# Results Analysis

In [ ]:
data_carla = get_runs("biermann-carla-university-of-cambridge","L65-project", "insertion_sort")

Found 106 runs.
Skipping run eaegztbg with algorithm ['find_maximum_subarray_kadane'] and steps 0
Skipping run h051r8qz with algorithm ['find_maximum_subarray_kadane'] and steps 0
Skipping run o05bop9e with algorithm ['find_maximum_subarray_kadane'] and steps 0
Skipping run x523mjg6 with algorithm ['find_maximum_subarray_kadane'] and steps 0
Skipping run 36gc4ukd with algorithm ['find_maximum_subarray_kadane'] and steps 0
Skipping run fjubtgaj with algorithm ['find_maximum_subarray_kadane'] and steps 0
Skipping run otit1iji with algorithm ['find_maximum_subarray_kadane'] and steps 0
Skipping run q2reu2oz with algorithm ['find_maximum_subarray_kadane'] and steps 0
Skipping run 0r51j0ll with algorithm ['find_maximum_subarray_kadane'] and steps 0
Skipping run 2kvcesxt with algorithm ['find_maximum_subarray_kadane'] and steps 0
Skipping run 7gdz2f4d with algorithm ['find_maximum_subarray_kadane'] and steps 0
Skipping run rct3f7a8 with algorithm ['find_maximum_subarray_kadane'] and steps 49

In [57]:
len(data_carla)

0